In [ ]:
import os
import re
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_chroma import Chroma
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document


In [2]:
def open_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            contents = file.read()
        return contents
    except FileNotFoundError:
        print("File not found:")
    except Exception as e:
        print("Error:")

In [3]:
def inferir_metadados(texto, filename):
    match = re.search(r"(Parte\s+[IVXL]+:.*?)\n", texto)
    return {
        "nome do arquivo": filename,
        "seção": match.group(1) if match else "Desconhecida"
    }

In [ ]:
client = Ollama(model="llama3:8b")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#client = OpenAI(api_key=OPENAI_API_KEY)

embedding = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

chunk_size = 1300 
chunk_overlap = 200
file_path = "documents/OAC-Resumo.txt"
persist_directory = "chroma-oac"

criar_db = False

C:\Users\lucca\AppData\Local\Temp\ipykernel_34608\4038600018.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  client = Ollama(model="llama3:8b")
C:\Users\lucca\AppData\Local\Temp\ipykernel_34608\4038600018.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


In [5]:
if criar_db:
    print("Criando Banco Vetorial")

    # Abrindo o arquivo
    texto = open_file(file_path)
    filename = os.path.basename(file_path)
    metadatas = [{"nome do arquivo": filename}]

    text_splitter = RecursiveCharacterTextSplitter(
                                        chunk_size=chunk_size,
                                        chunk_overlap= chunk_overlap,
                                        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
                                        )


    documentos = []
    for chunk in text_splitter.split_text(texto):
        metadados = inferir_metadados(chunk, filename)
        documentos.append(Document(page_content=chunk, metadata=metadados))

    vectorstore = Chroma.from_documents(documents=documentos, embedding=embedding, persist_directory=persist_directory)


else:
    print("Carregando Banco Vetorial")
    try:
        vectorstore = Chroma(embedding_function=embedding, persist_directory=persist_directory)
        print("Banco de vetores carregado com sucesso.")
    except Exception as e:
        print(f"Erro ao carregar banco de vetores: {e}")

Carregando Banco Vetorial
Banco de vetores carregado com sucesso.


In [11]:
question = "Qual a definição de um pipeline?"

retriever  = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

docs = retriever.get_relevant_documents(question)

docs_scores = vectorstore.similarity_search_with_score(question, k=6)
for doc, score in docs_scores:
    print(f"\n📄 Score: {score:.4f}")
    print(doc.page_content[:300])


📄 Score: 0.3450
Estrutura do Processador: Composta pela ALU, unidade de controle e registradores. O ciclo de instrução (buscar, decodificar, executar) é realizado por uma sequência de micro-operações atômicas.
Pipeline: A eficiência do pipeline, no entanto, pode ser degradada por hazards (estruturais, de dados e de

📄 Score: 0.3677
2. Questões de Desempenho
Projeto para Desempenho: A otimização do desempenho envolve um esforço contínuo para aumentar a taxa de execução de instruções. As principais técnicas incluem:
Pipeline: Sobrepõe a execução de múltiplas instruções em diferentes estágios, como uma linha de montagem.
Previsão

📄 Score: 0.4039
RISC vs. CISC: A filosofia CISC busca transferir a complexidade do software para o hardware, tentando criar instruções que se assemelhem a operações de linguagens de alto nível. Em contrapartida, a filosofia RISC transfere a complexidade do hardware para o software (o compilador). Ao fornecer um con

📄 Score: 0.4086
Lei de Little: Um princípio f

In [7]:
def rag_consultas(question):

    context = retriever.get_relevant_documents(question)

    prompt_template = PromptTemplate(

        input_variables=["context", "question"],

        template="""
            
            Você é um assistente especialista em Organização e Arquitetura de Computadores com o objetivo de auxiliar estudantes da disciplina a entenderem o conteúdo.

            Baseie sua resposta **exclusivamente** no texto abaixo. Sua resposta deve ser gerada em **formato Markdown**, clara, objetiva e adequada para **estudantes iniciantes**. Para questões técnicass, caso a pergunta solicitada não esteja em conformidade com o texto sugerido, expor o texto que foi enviado para referência.

            Inclua sempre ao final:
            - Uma **avaliação percentual** de quanto o texto de apoio foi essencial para responder à pergunta.
            - Uma explicação breve indicando **quais trechos do texto** foram mais coerentes com a resposta.

            Responda sempre em **português do Brasil**, usando linguagem acessível e, quando necessário, explique os termos técnicos com analogias ou exemplos simples.

            ---

            ### 📘 Texto de apoio:
            {context}

            ---

            ### ❓ Pergunta:
            {question}

            ---

            ### ✅ Resposta (em Markdown):

            """

    )

    prompt = prompt_template.format(context=context, question=question)

    """
    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    """

    result = client.invoke(prompt)

    return result
    
    
resposta = rag_consultas(question)

print("Resposta:", resposta)

Resposta: **Resposta:**
**Taxonomia de Flynn e categorias de sistemas computacionais**

A Taxonomia de Flynn é uma forma comum de categorizar sistemas com capacidade de processamento paralelo, proposta por Michael J. Flynn em 1972. A taxonomia define quatro categorias de sistemas computacionais baseadas na estrutura do sistema e na natureza da instrução:

* **SISD (Single Instruction, Single Data)**: um processador único executa uma única sequência de instruções para operar os dados armazenados em uma única memória. Uniprocessadores enquadram-se nessa categoria.
* **SIMD (Single Instruction, Multiple Data)**: uma única instrução de máquina controla a execução simultânea de uma série de elementos de processamento em operações básicas. Cada elemento de processamento possui uma memória de dados associada, então cada instrução é executada em um conjunto diferente de dados por processadores diferentes. Processadores de vetores e matrizes se enquadram nessa categoria.
* **MISD (Multiple Inst